# Amazon Fine Food Review Analysis

Data source : https://www.kaggle.com/snap/amazon-fine-food-reviews

This dataset consists of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.
Contents

    Reviews.csv: Pulled from the corresponding SQLite table named Reviews in database.sqlite
    database.sqlite: Contains the table 'Reviews'

Data includes:

    Reviews from 
    Number of reviews : 568,454
    Number of Users : 256,059 users
    Number of Products : 74,258 products
    Timespan : Oct 1999 - Oct 2012
    Number of Attributes/Columns : 10
    260 users with > 50 reviews
    
Attribute Information :

    1. Id
    2. ProductId : Unique identifier for the product
    3. UserId : Unqiue identifier for the user
    4. ProfileName : Profile name of the user
    5. HelpfulnessNumerator : Number of users who found the review helpful
    6. HelpfulnessDenominator : Number of users who indicated whether they found the review helpful or not
    7. Score : Rating between 1 and 5
    8. Time : Timestamp for the review
    9. Summary : Brief summary of the review
    10. Text : Text of the review
   

## Loading the Dataset 
After Downloading the data we can see that the data is in sqlite format which will help us to make query and visualisation simple and easy.
So first Since here we have 10 attribute. So for analysis and prediction purpose we are creating another attribute in place of score attribute by simply classifying the positive scores (Score 4 and score 5) and Negative Scores (Score 1 and score 2) and will ignore the score level of 3. 

In [17]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import scipy


In [18]:
connect=sqlite3.connect('database.sqlite')

#Filtering only positive and negative reviews i.e. just avoiding taking reviews having score =3
filtered_data=pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score!=3""",connect)

#for filteration as positive and negative 

def partition(x):
    if x>3:
        return 'positive'
    return 'negative'

#now putting change into the table
actual_score=filtered_data['Score']
positivenegative=actual_score.map(partition)
filtered_data['Score']=positivenegative



In [19]:
filtered_data.shape

(525814, 10)

In [20]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Exploratary Data Analysis

#### Data Cleaning : Duplication
As we have seen the data, this data contains many duplicate entries. As many discription of this data set we have come to know that this

In [21]:
display_data=pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score!=3 AND UserId='AR5J8UI46CURR' ORDER BY ProductID""", connect);
display_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [22]:
#sorting the data according to ProductID in ascending order
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True)
sorted_data.shape

(525814, 10)

In [23]:
#Deduplication of entries
final_data=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep="first",inplace=False)
final_data.shape

(364173, 10)

In [24]:
#Removing some errored data
final_data=final_data[final_data.HelpfulnessNumerator<=final_data.HelpfulnessDenominator]
final_data.shape

(364171, 10)

In [25]:
#Now just to Count how many positive and negative reviews are present
final_data['Score'].value_counts()

positive    307061
negative     57110
Name: Score, dtype: int64

## Text Preprocessing
As we can see through the data the most important attribute is text attributes i.e. Summery and Text. So if we are able to convert anything to vector then we can perform linear algebra stuff with the data. 
* So if we are able to convert the text part into the n dimension vector then we can perform many linear algebra things with that

### Bag of Word (BoW):
    1: Constructing a dictionary : Constructing set of all the words
    2: Constructing a matrix which represent each of the word of the dictionary. \
        -> Constructing a vector of size of the dictinary (say d) where each word is a different dimension. 
        -> Where each cell of the vector represent the occurance of that particular word.
        -> So we can generate a vector of size n (dictionary sized) for each review where it can be sparse matrix.(most of its values will be  zero)
 
    With the help of BoW, we can able to construct some kind of relation between each of the reviews. Similiar text will result closer vector in d dimension.


In [26]:
#Bag-of-Words
count_vect=CountVectorizer() #in scikit-learn
final_count=count_vect.fit_transform(final_data['Text'].values)
#print(final_count)
#final_count is the sparse matrix

In [27]:
print(type(final_count))
print(final_count.shape)


#print(scipy.sparse.csr_matrix.todense(final_count))


<class 'scipy.sparse.csr.csr_matrix'>
(364171, 115281)


Here we can see that it converts to values such that there are 115281 columns each representing a distinct word.

### Stemming, Stop Words removing and lemmatization
* Removing the html tags
* Removing pancuations or special charector like or, #...
* check if the word is made up of english language or alphanumeric
* check to see if length of the word greater then 2.
* convert the words to lower case
* remove stop words
* Final Snowball stemming the word

In [28]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop=set(stopwords.words('english'))
#print(stop)
sno=nltk.stem.SnowballStemmer('english')

#clean the data tags
def clean_tags(data):
    clean_val=re.compile('<.*?>')
    clean_text=re.sub(clean_val, ' ', data)
    return clean_text
def clean_panctuations(data):
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',data)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned
#Stemming
print(sno.stem("Greetings"))

greet


In [29]:
#print((final_data['Score'].values[900]))

In [30]:
i=0
str1=''
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
#'''''''
for sent in final_data['Text'].values:
    filtered_sentences=[]
    sent=clean_tags(sent)
    
    
    for w in sent.split():
        for cleaned_words in clean_panctuations(w).split():
            if ((cleaned_words.isalpha())& (len(cleaned_words)>2)):
                if (cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower()).encode('utf8'))
                    filtered_sentences.append(s)
                    if (final_data['Score'].values)[i]=='positive':
                        all_positive_words.append(s)
                    if (final_data['Score'].values)[i]=='negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    #final string after cleaning
    string1= b" ".join(filtered_sentences)
    final_string.append(string1)
    i=i+1
                
#print(final_string)                
               
#'''

In [31]:
final_data['cleanned_text']=final_string

In [32]:
final_data.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleanned_text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...


In [33]:
#store final table into SQLLite table for future
conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final_data.to_sql('Reviews',conn,schema=None,if_exists='replace')

### Unigrams, Bigrams and n Grams:
#### Unigrams:
    divide the whole sentence into 1 dimension words. Or in other words each word is considered a dimension. But in this case we may loose sequential information.
#### Bi-grams:
    Pairs of words is considered as a dimension.
#### Tri-grams:
    3- consecutive words are considered as a dimension.




In [34]:
freq_dist_pos=nltk.FreqDist(all_positive_words)
freq_dist_neg=nltk.FreqDist(all_negative_words)

In [35]:
print('Top 10 common positive words : ',freq_dist_pos.most_common(10))
print('\nTop 10 common negative words: ',freq_dist_neg.most_common(10))

Top 10 common positive words :  [(b'like', 139429), (b'tast', 129047), (b'good', 112766), (b'flavor', 109624), (b'love', 107357), (b'use', 103888), (b'great', 103870), (b'one', 96726), (b'product', 91033), (b'tri', 86791)]

Top 10 common negative words:  [(b'tast', 34585), (b'like', 32330), (b'product', 28218), (b'one', 20569), (b'flavor', 19575), (b'would', 17972), (b'tri', 17753), (b'use', 15302), (b'good', 15041), (b'coffe', 14716)]


In [36]:
### Bigram
bigram_obj=CountVectorizer(ngram_range=(2,2))
bigram_count=bigram_obj.fit_transform(final_data['Text'].values)

In [37]:
bigram_count.get_shape()


(364171, 2794911)

### Tf-idf (Term frequency and Inverse Document Frequency)
#### Term Frequency:
    The probability of finding a word in a given document
#### Inverse Document Frequency: 

    log(N/ni)
* N = total number of documents in corpus
* ni = total number of documents in which the word present
* (N/ni)>1 
* and log(1)=0
* the value of idf will always >=1

The log of the total numbers of documents in the corpus devided by number of document that contains the word. So this value will always greater than 0.


* if the word is more frequent then idf will be low
* and if the word is rare frequent then idf will be high

So in Tf-idf we multiply the value of tf and idf for each word and put that value corospond to the vector value in each documents sparse vector.

The only disadvantage of the tf-idf is that it not takes the sementic meaning


In [38]:
tf_idf_obj=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_obj.fit_transform(final_data['Text'].values)

In [39]:
print(final_tf_idf.get_shape())
feature_names=tf_idf_obj.get_feature_names()
print(len(feature_names))

(364171, 2910192)
2910192


In [40]:
print(feature_names[100000:100010])

['ales until', 'ales ve', 'ales would', 'ales you', 'alessandra', 'alessandra ambrosia', 'alessi', 'alessi added', 'alessi also', 'alessi and']


In [41]:
#to get top tf_idf values:
#source : https://buhrmann.github.io/tfidf-analysis.html

def top_tf_idf_feat(raw,features,top_n):
    topn_ids=np.argsort(raw)[::-1][:top_n]
    #print(topn_ids)
    #type(topn_ids)
    top_feats = [(features[i], raw[i]) for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns=['Features','if_idf_score']
    return (df)

In [42]:
row=final_tf_idf[1,:].toarray()[0]
top_tf_idf=top_tf_idf_feat(row,feature_names,10)
print(top_tf_idf)

            Features  if_idf_score
0       sendak books      0.173437
1        rosie movie      0.173437
2    paperbacks seem      0.173437
3      cover version      0.173437
4       these sendak      0.173437
5     the paperbacks      0.173437
6         pages open      0.173437
7       really rosie      0.168074
8  incorporates them      0.168074
9         paperbacks      0.168074


### Word2Vec
It preserves the sementic meaning as well as their relationships (actual meaning like men and women vs kind and queen). It didn't store terms as sparse matrix.If W1 and W2 are semantically similiar then its vector v1 and v2 will be closed. (Vman-Vwoman) is parallel to (Vkind-Vqueen). W2V is learning the semantic automatically by raw text. The larger the dimension tends to more more information in W2V algorithm. This is really very powerfull Technique
* The larger dimension tends to more information rich vector we have. 



In [43]:

from gensim.models import Word2Vec

In [44]:
import gensim
i=0
list_of_sentences=[]
for sent in final_data['Text'].values:
    filtered_sentence=[]
    sent=clean_tags(sent)
    for w in sent.split():
        for cleaned_words in clean_panctuations(w).split():
            if (cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sentences.append(filtered_sentence)
    

In [45]:
w2v_model=gensim.models.Word2Vec(list_of_sentences,min_count=5,size=50,workers=4)

In [48]:
words=list(w2v_model.wv.vocab)
print(len(words))
#w2v_model.wv.most_similar('tasty')
words[0:5]

33783


['this', 'witty', 'little', 'book', 'makes']

In [47]:
w2v_model.wv.most_similar('like')

[('resemble', 0.7106888890266418),
 ('dislike', 0.647260844707489),
 ('mean', 0.643530547618866),
 ('prefer', 0.6375082731246948),
 ('think', 0.6203250885009766),
 ('overpower', 0.6108693480491638),
 ('overwhelm', 0.6095193028450012),
 ('enjoy', 0.6023695468902588),
 ('bother', 0.5895633697509766),
 ('miss', 0.5820777416229248)]

#### Average Word2Vec:
    Average Word2Vec is the average of all the Word2Vec for each word in a given document. It is used to Calculate Word2Vec for sentences
                 
                 Average Word2Vec = 1/n(Word2Vec(w1)+Word2Vec(W2)+........)
#### Tf-idf-Word2Vec:
    Simple weighting stretegies to convert sentences into vecFirst it compute Tf-idf for document r1 and return a vector for document r1. And simulteneuosly calculate Word2Vec for each of the word and multiply with that document vector's word index.
        
      Tf-idf-Word2Vec(w1) = (t1*Word2Vec(w1)+t2*Word2Vec(w2)+...)/t1+t2+... , Where ti is the tfidf value of wi word
      Tf-idf-Word2Vec(sentence)= (ti*Word2Vec(wi))/(ti)

In [53]:
print((list_of_sentences[1]))

['i', 'grew', 'up', 'reading', 'these', 'sendak', 'books', 'and', 'watching', 'the', 'really', 'rosie', 'movie', 'that', 'incorporates', 'them', 'and', 'love', 'them', 'my', 'son', 'loves', 'them', 'too', 'i', 'do', 'however', 'miss', 'the', 'hard', 'cover', 'version', 'the', 'paperbacks', 'seem', 'kind', 'of', 'flimsy', 'and', 'it', 'takes', 'two', 'hands', 'to', 'keep', 'the', 'pages', 'open']


In [59]:
w2v_model.wv.most_similar('hey')

[('yeah', 0.7761039733886719),
 ('lol', 0.7219513654708862),
 ('uh', 0.7139227390289307),
 ('stupid', 0.7081554532051086),
 ('seriously', 0.6896477937698364),
 ('heck', 0.6497436761856079),
 ('kidding', 0.6401499509811401),
 ('yep', 0.6397970914840698),
 ('damn', 0.6382590532302856),
 ('anyways', 0.6269793510437012)]

#### Calculating Word2Vec for each sentences in review



In [61]:
sent_vector=[]
for sent in list_of_sentences:
    sent_vec=np.zeros(50)
    count_words=0
    for word in sent:
        try:
            vec=w2v_model.wv[word]
            sent_vec+=vec
            count_words+=1
        except:
            pass
    sent_vec=sent_vec/count_words
    sent_vector.append(sent_vec)

print(len(sent_vector))
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


364171


In [71]:
print((list_of_sentences[1]))
print((sent_vector[1]))

['i', 'grew', 'up', 'reading', 'these', 'sendak', 'books', 'and', 'watching', 'the', 'really', 'rosie', 'movie', 'that', 'incorporates', 'them', 'and', 'love', 'them', 'my', 'son', 'loves', 'them', 'too', 'i', 'do', 'however', 'miss', 'the', 'hard', 'cover', 'version', 'the', 'paperbacks', 'seem', 'kind', 'of', 'flimsy', 'and', 'it', 'takes', 'two', 'hands', 'to', 'keep', 'the', 'pages', 'open']
[-0.30392123 -0.79201868  0.33612066 -0.35314329 -0.52123807 -0.13319498
 -0.60553056 -0.69415923 -0.05810921 -0.19672782 -0.68593011  0.8393152
 -0.43446464 -0.24793821  0.61800498 -0.0364209   0.05479705  1.14821459
 -0.65693565 -0.30334259 -1.00440464 -0.0327446  -0.2563789   1.31766732
 -0.37717317 -0.37804507  1.33098585 -0.44040332  0.34808814 -0.4387633
  0.31092072  0.16986976 -0.44022827  0.09674198  0.12834727  0.76208363
  0.27461769 -0.12432189  0.3808894   1.20819422 -0.14434112 -0.00211157
  0.42197797 -1.00044272 -0.63553785  1.08340417 -0.47544294 -0.06401876
 -0.73018969  0.258

#### Going towards tf-idf-word2Vec

In [73]:
tf_idf_features=tf_idf_obj.get_feature_names()

In [75]:
print(len(tf_idf_features))

2910192


In [76]:
print(final_tf_idf)

  (0, 1268863)	0.08275523980718573
  (0, 1322643)	0.05736320423975409
  (0, 1181493)	0.05904930622559946
  (0, 2815806)	0.06353114992572337
  (0, 1562605)	0.12941275579745923
  (0, 1032033)	0.11573055882069516
  (0, 2075535)	0.12941275579745923
  (0, 2616462)	0.12541096971801555
  (0, 49126)	0.04475667962178877
  (0, 283308)	0.054444217381069276
  (0, 2381603)	0.0769491583240197
  (0, 2837880)	0.07781013221409352
  (0, 2324657)	0.0937743722525583
  (0, 324043)	0.11573055882069516
  (0, 2608442)	0.10301992784348538
  (0, 2838349)	0.06715422842681543
  (0, 126268)	0.0893377308667213
  (0, 361978)	0.11856987122963351
  (0, 552188)	0.12941275579745923
  (0, 1319489)	0.0927612148772383
  (0, 2578785)	0.026787698091940104
  (0, 105321)	0.06840813514421003
  (0, 1333169)	0.053575946913953454
  (0, 1739789)	0.046544793491389236
  (0, 2255427)	0.12941275579745923
  :	:
  (364170, 1254867)	0.03283894855017248
  (364170, 2614666)	0.05504239102854844
  (364170, 2897041)	0.040848095932348116
  (364

In [86]:
tf_idf_w2v_sent_vector=[]
row=0
for sent in list_of_sentences:
    sent_vect=np.zeros(50)
    tf_idf_sum=0
    for word in sent:
        try:
            vec=w2v_model.wv[word]
            tf_idf_word=final_tf_idf[row,tf_idf_feature.index(word)]
            sent_vect+=(vec*tf_idf_word)
            tf_idf_sum+=tf_idf_word
        except:
            pass
    sent_vect=sent_vect/tf_idf_sum
    tf_idf_w2v_sent_vector.append(sent_vec)
    row+=1

print(len(tf_idf_w2v_sent_vector))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


364171


In [89]:
print(tf_idf_w2v_sent_vector[1:3])

[array([-0.26661966, -0.03295791, -0.25479811, -0.46635248,  0.34355303,
       -0.39239626, -0.68777127, -0.88658103,  0.1324354 ,  0.07070138,
       -0.35070344,  0.81264022, -0.53939053, -0.09726452,  0.57552282,
       -0.41481607,  0.47379984,  1.75258809, -0.46194684,  0.59258366,
       -0.78632025,  0.34842148, -0.28700296,  0.86899327, -0.50096115,
       -0.19244379,  1.24692652, -0.7663961 ,  0.62684289,  0.26949071,
        0.0308574 ,  0.43166185,  0.10855806, -0.40827156,  0.45064414,
        1.13380192,  0.35173921,  0.78460497,  0.55599871,  1.46632871,
       -2.08668473, -0.10661922,  0.50243684, -0.49826203, -0.88207639,
        0.78687283, -1.03372064,  0.17053629, -0.45100903,  1.34717095]), array([-0.26661966, -0.03295791, -0.25479811, -0.46635248,  0.34355303,
       -0.39239626, -0.68777127, -0.88658103,  0.1324354 ,  0.07070138,
       -0.35070344,  0.81264022, -0.53939053, -0.09726452,  0.57552282,
       -0.41481607,  0.47379984,  1.75258809, -0.46194684,  0